In [ ]:
import pandas as pd
import numpy as np
from sqlalchemy import create_engine

import re

# --- CONFIGURATION ---
DB_USER = 'root'
DB_PASS = 'Echidna12'
DB_HOST = 'localhost'
DB_PORT = '3306'
DB_NAME = 'heidi_test'

# Setup Database Connection
connection_str = f"mysql+pymysql://{DB_USER}:{DB_PASS}@{DB_HOST}:{DB_PORT}/{DB_NAME}"
engine = create_engine(connection_str)

In [ ]:
# --- 1. LOAD DATA ---
csv_path = r"C:\Users\Deyvi\Documents\InduMine\data\grouped_products_final.csv"
print("Loading CSV...")
df = pd.read_csv(csv_path, dtype=str, encoding='utf-8-sig', low_memory=False)

In [ ]:
# Rename Columns
rename_map = {
    "Application method": "Application method.1",
    "Application surface": "Application surface.1",
    "Degree of protection": "Degree of protection.1",
    "Depth": "Depth.1",
    "Height": "Height.1",
    "Number of poles": "Number of poles.1",
    "Rated current": "Rated current.1",
    "Rated power": "Rated power.1",
    "Smart code": "Smart code.1",
    "Surface preparation": "Surface preparation.1",
    "Weight": "Weight.1",
    "Width": "Width.1"
}
df.rename(columns=rename_map, inplace=True)

In [ ]:
# --- 2. EXTRACT URL CATEGORIES ---
def extract_category(url):
    if pd.isna(url): return None
    s = str(url)
    parts = s.split('/')
    # Logic: AfterDelimiter(_, "/", 6) -> Replace "//"
    if len(parts) > 7:
        remaining = "/".join(parts[7:]).replace("//", "")
        sub_parts = remaining.split('/')
        
        # Product URL.1 (Key), Product URL.2 (Category), Product URL.3 (SubCategory)
        p1 = sub_parts[0] if len(sub_parts) > 0 else None
        p2 = sub_parts[1] if len(sub_parts) > 1 else None
        p3 = sub_parts[2] if len(sub_parts) > 2 else None
        return pd.Series([p1, p2, p3])
    return pd.Series([None, None, None])

print("Processing URLs...")
df[['Product URL.1', 'Product URL.2', 'Product URL.3']] = df['Product URL'].apply(extract_category)

In [ ]:
# --- 3. FILTERING ---
print("Filtering excluded categories...")

# List of prefixes to exclude
exclusions = [
    "contactor", 
    "emergency", 
    "gateway", 
    "motor-protector",
    "simultaneity", 
    "zero-speed" 
]

# Create a filter mask (True for rows we want to KEEP)
# "not Text.StartsWith" equivalent
mask = ~df['Product URL.1'].str.lower().str.strip().str.startswith(tuple(exclusions), na=False)

# Apply the filter
df = df[mask]

In [ ]:
# --- 4. CLEAN GROUP NAMES ---
# Prepare the grouping key (remove hyphens, etc.)
df['Product URL.1'] = df['Product URL.1'].str.replace("-", " ", regex=False)
df['Product URL.1'] = df['Product URL.1'].str.replace("%26", "", regex=False)
df['Product URL.1'] = df['Product URL.1'].str.replace("%2C", " ", regex=False)
df['Product URL.1'] = df['Product URL.1'].str.replace(r"\s+", " ", regex=True).str.strip()

In [ ]:
# --- 5. DEFINE CLEANING FUNCTION ---
def clean_group_data(group_df):
    # A. Remove grouping column
    group_df = group_df.drop(columns=['Product URL.1'], errors='ignore')
    
    # B. Remove columns that are all empty/null/(blank)
    def is_col_valid(series):
        invalid_values = [None, "", "(blank)", np.nan]
        return not series.isin(invalid_values).all()

    valid_cols = [c for c in group_df.columns if is_col_valid(group_df[c])]
    group_df = group_df[valid_cols]
    
    # C. Rename Category columns if they exist
    col_renames = {}
    if "Product URL.2" in group_df.columns: col_renames["Product URL.2"] = "Category"
    if "Product URL.3" in group_df.columns: col_renames["Product URL.3"] = "SubCategory"
    group_df.rename(columns=col_renames, inplace=True)
    
    # D. Clean text inside Category/SubCategory
    cols_to_clean = [c for c in ["Category", "SubCategory"] if c in group_df.columns]
    for col in cols_to_clean:
        group_df[col] = group_df[col].astype(str)
        group_df[col] = group_df[col].str.replace("-", " ", regex=False)
        group_df[col] = group_df[col].str.replace("%2C", "", regex=False)
        group_df[col] = group_df[col].str.replace("%28", "", regex=False)
        group_df[col] = group_df[col].str.replace("%29", "", regex=False)
    
    return group_df

In [ ]:
# --- 6. LOOP AND EXPORT ---
print("Grouping and Exporting to SQL...")

groups = df.groupby("Product URL.1")

for group_name, group_data in groups:
    if not group_name or str(group_name).lower() == 'nan':
        continue

    # Clean the specific table
    cleaned_table = clean_group_data(group_data.copy())
    
    # Generate Table Name (lowercase, no spaces)
    table_name = str(group_name).lower()
    table_name = re.sub(r'[^a-z0-9]', '_', table_name)
    table_name = re.sub(r'_+', '_', table_name).strip('_')
    table_name = table_name[:60] # Truncate to be safe for MySQL
    
    print(f"-> Exporting table: {table_name} ({len(cleaned_table)} rows)")
    
    try:
        cleaned_table.to_sql(name=table_name, con=engine, if_exists='replace', index=False)
    except Exception as e:
        print(f"   Error exporting {table_name}: {e}")

print("Done!")

Loading CSV...
Processing URLs...
Filtering excluded categories...
Grouping and Exporting to SQL...
-> Exporting table: building_infrastructure (25 rows)
-> Exporting table: coatings_and_varnishes (177 rows)
-> Exporting table: critical_power (9 rows)
-> Exporting table: digital_solutions (2 rows)
-> Exporting table: digital_solutions_and_smart_grid (51 rows)
-> Exporting table: electric_motors (95 rows)
-> Exporting table: generation_transmission_and_distribution (8 rows)
-> Exporting table: industrial_automation (1027 rows)
-> Exporting table: safety_industrial_sensors_and_power_supply (219 rows)
Done!
